In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from my_utils import load_data
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense, Dropout
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

In [3]:
X, session_df, requests_df = load_data(log_path='../dataset/output.log', req_thres=4, normalize_feat=True)

LOADING THE DATASET...


Skipping line 601729: Expected 7 fields in line 601729, saw 23. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.


DATASET HAS BEEN LOADED SUCESSFULLY!


In [18]:
paths = requests_df.groupby(['ip', 'user_agent'])['path'].transform(lambda x: ' '.join(x))
requests_df['merged_path'] = paths

,ip,time,method,status_code,path,response_length,user_agent,response_time,path_count_normalized,path_length,time_stamp,merged_path
0,207.213.193.143,2021-05-12 05:06:00+04:30,Get,304,cdn/profiles/1026106239,0,Googlebot-Image/1.0,32.0,0.011434,3,1620779760,cdn/profiles/1026106239 images/badge.png cdn/p...
1,207.213.193.143,2021-05-12 05:06:00+04:30,Get,304,images/badge.png,0,Googlebot-Image/1.0,4.0,1.009145,2,1620779760,cdn/profiles/1026106239 images/badge.png cdn/p...
2,35.110.222.153,2021-05-12 05:06:00+04:30,Get,200,pages/630180847,52567,Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-...,32.0,0.101132,2,1620779760,pages/630180847 images/sanjagh_logo_purpule5.p...
3,35.108.208.99,2021-05-12 05:06:00+04:30,Get,200,images/fav_icon2.ico,23531,Mozilla/5.0 (Linux; Android 6.0; CAM-L21) Appl...,20.0,1.508523,2,1620779760,images/fav_icon2.ico images/cooler_service_ban...
4,35.110.222.153,2021-05-12 05:06:00+04:30,Get,200,images/sanjagh_logo_purpule5.png,4680,Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-...,8.0,2.218701,2,1620779760,pages/630180847 images/sanjagh_logo_purpule5.p...
...,...,...,...,...,...,...,...,...,...,...,...,...
1260028,35.117.86.75,2021-05-12 15:08:59+04:30,Get,304,images/graystar_min.png,0,Mozilla/5.0 (Linux; Android 9; Redmi 7A) Apple...,4.0,1.068165,2,1620815939,840087298/532572592 css/index.d778f9d4b03d27e4...
1260029,35.117.86.75,2021-05-12 15:08:59+04:30,Get,304,fonts/sanjagh_icon_font_5.woff,0,Mozilla/5.0 (Linux; Android 9; Redmi 7A) Apple...,4.0,2.326355,2,1620815939,840087298/532572592 css/index.d778f9d4b03d27e4...
1260030,153.126.251.199,2021-05-12 15:08:59+04:30,Get,101,api/v2/connect/215865643,0,okhttp/3.12.1,60003.0,0.000081,4,1620815939,cdn/servicePic/1324279420 cdn/servicePic/47004...
1260031,207.213.207.102,2021-05-12 15:08:59+04:30,Get,304,cdn/profiles/1289255230,0,Googlebot-Image/1.0,20.0,0.000564,3,1620815939,cdn/articles/1148001967 cdn/pro_photo_gallery/...


In [22]:
path_df = requests_df.set_index(['ip', 'user_agent'])
path_df = path_df['merged_path']

path_df = pd.DataFrame(path_df, columns=['merged_path'])
path_df

merged_path
ip              user_agent                                                                                           
207.213.193.143 Googlebot-Image/1.0                                 cdn/profiles/1026106239 images/badge.png cdn/p...
                Googlebot-Image/1.0                                 cdn/profiles/1026106239 images/badge.png cdn/p...
35.110.222.153  Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-J...  pages/630180847 images/sanjagh_logo_purpule5.p...
35.108.208.99   Mozilla/5.0 (Linux; Android 6.0; CAM-L21) Apple...  images/fav_icon2.ico images/cooler_service_ban...
35.110.222.153  Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-J...  pages/630180847 images/sanjagh_logo_purpule5.p...
...                                                                                                               ...
35.117.86.75    Mozilla/5.0 (Linux; Android 9; Redmi 7A) AppleW...  840087298/532572592 css/index.d778f9d4b03d27e4...
                Mozilla/5.0 (Linux; Android 9; Redmi 7A) AppleW...  840087298/532572592 css/index.d778f9d4b03d27e4...
153.126.251.199 okhttp/3.12.1                                       cdn/servicePic/1324279420 cdn/servicePic/47004...
207.213.207.102 Googlebot-Image/1.0                                 cdn/articles/1148001967 cdn/pro_photo_gallery/...
207.213.207.224 Googlebot-Image/1.0                                 cdn/profiles/2116702757 cdn/profiles/710004701...

[1241943 rows x 1 columns]

# Preprocessing

In [ ]:
import re

path_df['merged_path'] = path_df['merged_path'].apply(lambda x : re.sub('[0-9]','', x))
path_df['merged_path'] = path_df['merged_path'].apply(lambda x : re.sub('[.]','', x))

path_df.iloc[0].merged_path